# DATA CLEANING

### JOB APPROVALS BIS

In [ ]:
import pandas as pd
import numpy as np

path = "C:\\Users\\dmehri\\Documents\\DATA\\Long Job Duration\\"

df = pd.read_csv(path + "Long Job Dur - Job Approvals - BIS.csv", low_memory=False)
df["Job Number"] = df["Job Number"].astype(str).map(str.strip)


print ("Only do anlaysis on NBs")
print ("Length before drop", len(df))
df["Job Type"] = df["Job Type"].astype(str).map(str.strip)
df = df[df["Job Type"] == "NB"]
print ("Length after drop", len(df))
print ("Drop pro-cert jobs")
df["Pro-Cert Description"] = df["Pro-Cert Description"].astype(str).map(str.strip)
df = df[df["Pro-Cert Description"] != "Professionally Certified Application"]
print ("Length after drop", len(df))

print ("Drop BSA jobs")
df["BSA Calendar Number"] = df["BSA Calendar Number"].astype(str).map(str.strip)
df = df[df["BSA Calendar Number"] == "nan"]
print ("Length after drop", len(df))

print ("Drop CPC jobs")
df["CPC Calendar Number"] = df["CPC Calendar Number"].astype(str).map(str.strip)
df = df[df["CPC Calendar Number"] == "nan"]
print ("Length after drop", len(df))


df = df.rename(columns={'D_DATE': 'App Process Date'})

#df["Job Type"] = df["Job Type"].astype(str).map(str.strip)
#df = df[df["Job Type"] == "NB"]
#print ("len after drop", len(df))

print ("Drop NAN in app process date")
print ("len before drop", len(df))
df = df.dropna(subset = ['App Process Date'])
print ("len after drop", len(df))



df['Plan Approval Date'] = pd.to_datetime(df['Plan Approval Date'])
#df['Plan Approval Date'] = df['Plan Approval Date'].apply(lambda x: x.date())
df['App Process Date'] = pd.to_datetime(df['App Process Date'])
#df['App Process Date'] = df['App Process Date'].apply(lambda x: x.date())

#print ("Subset for when appointments end in PENS BIS dataset, October 2018")
#end_time = str(10) + '-' + str(1) + '-' + str(2018)

#mask = ( (df['App Process Date'] <= end_time) )

#df = df.loc[mask]

#print ("length after date subset", len(df))


#df['Difference'] = (df['Plan Approval Date'] - df['App Process Date']).dt.days


#pd.bdate_range(df['Plan Approval Date'],df['App Process Date'], freq = 'B')

df["bussDays"] = np.busday_count( df['App Process Date'].values.astype('datetime64[D]'), df['Plan Approval Date'].values.astype('datetime64[D]'))

#df["bussDays"] = df["bussDays"].abs()
print ("drop buss days less than zero", len(df))
df = df[df["bussDays"] >= 0].reset_index(drop=True)
print ("len after drop", len(df))


print ("drop by job number, initial and approval date")
df = df.drop_duplicates(subset=['Job Number', 'Plan Approval Date', 'App Process Date']).reset_index(drop=True)
print ("len after drop", len(df))


numunique = list(set(df["Job Number"].tolist() ))
print (len(numunique))
print ("number of unique job numbers", len(numunique))


df["bussYears"] = df["bussDays"]/365.0


columns = sorted(df.columns)


#df["Filing Representative First Name"] = df["Filing Representative First Name"].astype(str).map(str.strip)
#df["Filing Representative First Name"] = df["Filing Representative First Name"].map(str.upper)
#df["Filing Representative Last Name"] = df["Filing Representative Last Name"].astype(str).map(str.strip)
#df["Filing Representative Last Name"] = df["Filing Representative Last Name"].map(str.upper)
#df["Filing Representative Name"] = df["Filing Representative First Name"] + " " + df["Filing Representative Last Name"]


#df["OT Description"] = df["OT Description"].astype(str).map(str.strip).map(str.upper)
#df["OT Description"] = df["OT Description"].str.replace('.','')
#df["OT Description"] = df["OT Description"].map(str.strip)


print ("Create stories category")

def StoriesCat(x):
    if x < 7:
        return "Low Rise"
    if x >= 7:
        return "High Rise"

df["Stories Category"] = df["Proposed Stories"].apply(StoriesCat)



print ()
print ("Work types")
#df = df.replace({'X': '1'}, regex=True)
#df = df.replace({'Y': '1'}, regex=True)


#COMMON WORK TYPES BETWEEN DOB NOW ANBD BIS
#Boiler
#Mechanical
#Plumbing
#Standpipe
#Construction Equipment
#General Construction
#Curb Cut


worktypesorig = ['Boiler Flag', 'Mechanical Flag',
       'Plumbing Flag', 'Standpipe Flag',
       'Construction Equipment Flag', 'General Construction Flag',
       'Curb Cut Flag' ]


for w in worktypesorig:
    df[w] = df[w].str.replace('X', '1')
    df[w] = df[w].str.replace('Y', '1')

df = df.fillna('0')

df = df.rename(columns={'Boiler Flag': 'Boiler', 
                        'Mechanical Flag': 'Mechanical',
                       'Plumbing Flag':'Plumbing',
                       'Standpipe Flag':'Standpipe',
                       'Construction Equipment Flag':'Construction Equipment',
                       'General Construction Flag':'General Construction',
                       'Curb Cut Flag':'Curb Cut'})



print ("Drop fields not relevant for NB analysis")
dropfields = ["Chute", "Fence", "Enlargement Flag (Yes/No)",
             "Enlargement SQFT", "Construction Material", "Existing Stories", "Existing Zoning Floor Area",
             "Existing Dwelling Units", "Estimated Job Cost", "Other Equipment Flag", "EQ Other Description", 
             "Supported Scaffold", "Sidewalk Shed", "Sidewalk Shed/Linear Feet", "Filing Representative Last Name",
             "Filing Representative First Name"]

df = df.drop(dropfields, 1)




print ("Clean occupancy")
occupancy = sorted(list(set(df["Proposed Occupancy Classification Description"].tolist() )))
print ("len of plan examiners", len(occupancy))
print (sorted(occupancy) )
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('  ', ' ')
#df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.split(":").str[0]
#df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.split("(").str[0]
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].map(str.strip)
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('EDUCATIONAL', 'EDUCATION')
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('INSTITUTIONAL', 'INSTITUTION')
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('RESIDENTIAL: 1 & 2 FAMILY HOUSES', 'RESIDENTIAL 1-2 FAMILY HOUSES')


df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('ASSEMBLY: EATING & DRINKING', 'ASSEMBLY')
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('ASSEMBLY: INDOOR SPORTS', 'ASSEMBLY')
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('ASSEMBLY: OTHER', 'ASSEMBLY')
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('ASSEMBLY: OUTDOORS', 'ASSEMBLY')
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('ASSEMBLY: THEATERS, CONCERT HALLS', 'ASSEMBLY')

df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('FACTORY/INDUSTRIAL: LOW HAZARD', 'FACTORY INDUSTRIAL')
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('FACTORY/INDUSTRIAL: MODERATE HAZARD', 'FACTORY INDUSTRIAL')

df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('HIGH HAZARD: ACCELERATED BURNING', 'HIGH HAZARD')
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('HIGH HAZARD: HEALTH', 'HIGH HAZARD')
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('HIGH HAZARD: SUPPORTS COMBUSTION', 'HIGH HAZARD')

df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('INSTITUTION: ASSISTED LIVING', 'INSTITUTION')
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('INSTITUTION: DAY CARE', 'INSTITUTION')
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('INSTITUTION: INCAPACITATED', 'INSTITUTION')
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('RESIDENTIAL APT HOUSE', 'RESIDENTIAL: APARTMENT HOUSES')

df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('RESIDENTIAL APT HOUSE', 'RESIDENTIAL: APARTMENT HOUSES')

df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('STORAGE (LOW HAZARD)', 'STORAGE')
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('STORAGE: LOW HAZARD', 'STORAGE')
df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('STORAGE: MODERATE HAZARD', 'STORAGE')

df['Proposed Occupancy Classification Description'] = df['Proposed Occupancy Classification Description'].str.replace('RESIDENTIAL (HOTELS)', 'RESIDENTIAL: HOTELS, DORMITORIES')


df["Applicant License Number"] = df["Applicant License Number"].astype(str).map(str.strip)
df["Applicant Name"] = df["Applicant First Name"] + " " + df["Applicant Last Name"]

print ()
occupancy = sorted(list(set(df["Proposed Occupancy Classification Description"].tolist() )))
print ("len of plan examiners", len(occupancy))
print (sorted(occupancy))


print ("drop business days less than zero", len(df))
df2= df.copy()
df2 = df2[df2["bussDays"] > 0]
#df2 = df2[df2["bussDays"] <1000]
print ("len after drop", len(df2))

#print ("Mean", df2["bussDays"].mean())
#print ("Standard deviation", df2["bussDays"].std())
#print ("Median", df2["bussDays"].median())
#print ()
#df2["bussDays"].describe()


print ("Plan exam name")
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].astype(str)
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace('.', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace('PLAN EXAMINER', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace('PLAN EXAM', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace('PLAN EXA', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace('PLAN EX', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace('S/C ACCESS', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace('S/C EXAM', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace('BRONX', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace('BX', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace('QUEENS', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(' QNS', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(' QN', ' ')


df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(' BORO COMM', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(' CALL 311', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(' CALL CTR 311', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(' CALL CENTER', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(' CALL CENTER', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace('/BUILD IT BACK', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace('/ BUILD IT BACK', ' ')


df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace('BROOKLYN', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(' BK', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(' (BKLYN)', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(' MAN', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(' S I', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(' S/C', ' ')
#df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(' ()', '')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(' - ', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(' LYN', ' ')

df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(' ONLY', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace('(', '')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace(')', '')


df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace('   ', ' ')
df['Plan Examiner BIS Name'] = df['Plan Examiner BIS Name'].str.replace('  ', ' ')

print ("DONE")





### Normalize Skewed Continus Variables

Estimated Fee

In [ ]:
print (len(df))
df = df[df["J_TOTAL_ESTIMATED_FEE"] < 1000000]
print (len(df))


df["EstFeeLogNorm"] = np.log(df["J_TOTAL_ESTIMATED_FEE"])
estfeedesc = df["EstFeeLogNorm"].describe()
estfeedesc[7]
df["EstFeeLogNorm"] = df["EstFeeLogNorm"]/estfeedesc[7]

df["EstFeeLogNorm"].describe()

Proposed Zoning Floor Area

In [ ]:
print (len(df))
df = df[df["Proposed Zoning Floor area"] < 1000000]
print (len(df))

df["ZoningFloorLogNorm"] = np.log(df["Proposed Zoning Floor area"] + 1)
estfeedesc = df["ZoningFloorLogNorm"].describe()
estfeedesc[7]
df["ZoningFloorLogNorm"] = df["ZoningFloorLogNorm"]/estfeedesc[7]

df["ZoningFloorLogNorm"].describe()

Dwelling Units

In [ ]:
df["Proposed Dwelling Units"] = df["Proposed Dwelling Units"].astype(int)
df["DwellUnitsLogNorm"] = np.log(df["Proposed Dwelling Units"] + 1)
estfeedesc = df["DwellUnitsLogNorm"].describe()
estfeedesc[7]
df["DwellUnitsLogNorm"] = df["DwellUnitsLogNorm"]/estfeedesc[7]
df["DwellUnitsLogNorm"].describe()

FAR

In [ ]:
print (len(df))
df = df[df["Floor Area Ratio (FAR)  Floor Area"] < 900000]
print (len(df))

df["FARLogNorm"] = np.log(df["Floor Area Ratio (FAR)  Floor Area"] +1 )
estfeedesc = df["FARLogNorm"].describe()
estfeedesc[7]
df["FARLogNorm"] = df["FARLogNorm"]/estfeedesc[7]
df["FARLogNorm"].describe()


In [ ]:
#df2.to_csv(path + "Long Job Duration BIS 2.csv", index=False)

df.to_csv(path + "test.csv", index=False)

#df.to_csv(path + "test.csv", index=False)

### PLAN EXAMS

In [ ]:
df2 = df.copy()

pd.set_option('chained_assignment', None)

da = pd.read_csv(path + "Long Duration Plan Exam Report Darius.csv", low_memory=False)
print (len(da))

da["J_JOB_NUMBER"] = da["J_JOB_NUMBER"].astype(str)

#da2 = da[["Examiner Name"]]
#da2["Exam Count"] = 1

#daG = da2.groupby(['Examiner Name']).sum()
#daG = daG.add_suffix('').reset_index()

dac = da[["J_JOB_NUMBER"]]
dac["Plan Exam Count"] = 1
dac = dac.groupby(['J_JOB_NUMBER']).sum()
dac = dac.add_suffix('').reset_index()


dac = dac.sort_values(by = 'Plan Exam Count', ascending=False).reset_index(drop=True)

dacDic = dac.set_index('J_JOB_NUMBER')['Plan Exam Count'].to_dict()

df2["Plan Exam Count"] = df2["Job Number"].map(dacDic)

print ("Total number appointments after grouped", dac["Plan Exam Count"].sum() )
print ("Total number of appointments merged into df", df2["Plan Exam Count"].sum() )

print ("Drop job filings with nan plan exam counts")
df2 = df2.dropna(subset = ['Plan Exam Count'])

print ("len after drop", len(df2))






### PLAN EXAMINER ANALYSIS

In [ ]:
"""
dp = df2[["Plan Examiner BIS Name", "Plan Exam Count"]]
dp["Job Count"] = 1

dp2 = (dp.groupby('Plan Examiner BIS Name').agg({'Job Count':'sum', 'Plan Exam Count': 'sum'}).reset_index().rename(columns={'Job_Number':'Job_Number_count'}) )

#dp = dp.groupby(['Plan Examiner BIS Name']).sum()
#dp = dp.add_suffix('').reset_index()
dp2["Avg Number Plan Exams"] = dp2["Plan Exam Count"]/dp2["Job Count"]

dp3 = (dp.groupby('Plan Examiner BIS Name').agg({'Plan Exam Count':'median'}).reset_index().rename(columns={'Job_Number':'Job_Number_count'}) )

dp3Dic = dp3.set_index('Plan Examiner BIS Name')['Plan Exam Count'].to_dict()

dp2["Median Number Plan Exams"] = dp2["Plan Examiner BIS Name"].map(dp3Dic)

dp2["Above Upper Quartile"] = "No"
dp2["Above One Standard Dev"] = "No"

for i in range(0, len(dp2)):
    if dp2["Median Number Plan Exams"][i] > 8:
        dp2["Above Upper Quartile"][i] = "Yes"
        
    if dp2["Avg Number Plan Exams"][i] > 10.7:
        dp2["Above One Standard Dev"][i] = "Yes"

print (df2["Plan Exam Count"].describe())

#dp.to_csv
dp2

"""

In [ ]:
#dp2.to_csv(path + "Plan Examiner Stats.csv", index=False)

In [ ]:
#da.to_csv(path + "test.csv", index=False)

In [ ]:
#df2.to_csv(path + "test.csv", index=False)


In [ ]:
#print (df2["Plan Exam Count"].describe())


In [ ]:
#df2.to_csv(path + "Long Job Duration BIS 2.csv", index=False)


In [ ]:
"""
df2 = df2.reset_index(drop=True)
countzeros = ['Proposed Stories', 'Proposed Zoning Floor area', 'BIN Number',
       'Lot Area (sqft)',
       'J_BALANCE_DUE', 'Difference', 'bussDays', 'bussYears',
       'Appointment Count',"Proposed Dwelling Units", "Lot Area (sqft)", "Floor Area Ratio (FAR)  Floor Area", 
        "J_TOTAL_ESTIMATED_FEE" ]

for p in countzeros:
    print (p)
    dfcountzeros = df2.copy()
    dfcountzeros = dfcountzeros[dfcountzeros[p] == 0.0]
    print ("percent zero value:", 100*( float(len(dfcountzeros))/len(df2) )    )
    print ()
    
#dfcountzeros = df2.copy()
#dfcountzeros = dfcountzeros[dfcountzeros["Lot Area (sqft)"] == 0.0].reset_index(drop=True)    
#print ("percent:", 100*( float(len(dfcountzeros))/len(df2) )    )
"""

In [ ]:
origzones = sorted(list(set(df2["Zoning District"] )))
#origzones

In [ ]:
df2 = df2.reset_index(drop=True)
df2["Zoning District"] = df2["Zoning District"].astype(str)
df2["Zoning District2"] = df2['Zoning District'].str.split("-").str[0]


zonedic = {'C1':'C_Local_Retail_Service',
          'C2':'C_Local_Retail_Service',
          'C3':'C_Waterfront_Recreation',
          'C3A':'C_Waterfront_Recreation',
          'C4':'C_Contextual_Non_Contextual_General',
          'C5':'C_Restricted',
          'C6':'C_General_Central',
          'C7':'C_Ammusement', 
          'C8':'C_General_Service',
          'M1':'M_Light',
          'M2':'M_Medium',
          'M3':'M_Heavy',
           
           
          'R1':'R_Low_Density_Non_Contextual',
           'R2':'R_Low_Density_Non_Contextual',
           'R2A':'R_Low_Density_Non_Contextual',
           'R2X':'R_Low_Density_Non_Contextual',
           'R3':'R_Low_Density_Non_Contextual',
           'R3A':'R_Low_Density_Non_Contextual',
           'R3X':'R_Low_Density_Non_Contextual',
           
           'R4':'R_Low_Density_Non_Contextual',
           'R4A':'R_Low_Density_Non_Contextual',
           'R4B':'R_Low_Density_Non_Contextual',
           
           'R5':'R_Low_Density_Non_Contextual',
           'R5A':'R_Low_Density_Non_Contextual',
           'R5B':'R_Low_Density_Non_Contextual',
           'R5D':'R_Low_Density_Non_Contextual',
           
           'R6':'R_Medium_Density_Non_Contextual',
           'R6A':'R_Medium_Density_Non_Contextual',
           'R6B':'R_Medium_Density_Non_Contextual',

          'R7':'R_Medium_Density_Non_Contextual',
           'R7A':'R_Medium_Density_Non_Contextual',
           'R7B':'R_Medium_Density_Non_Contextual',
           'R7D':'R_Medium_Density_Non_Contextual',
           'R7X':'R_Medium_Density_Non_Contextual',
           
           'R8':'R_Medium_Density_Non_Contextual',
           'R8A':'R_Medium_Density_Non_Contextual',
           'R8B':'R_Medium_Density_Non_Contextual',
           'R8X':'R_Medium_Density_Non_Contextual',
           
              
           'R9':'R_High_Density_Non_Contextual',
           'R9A':'R_High_Density_Non_Contextual',
           'R9D':'R_High_Density_Non_Contextual',
           'R9X':'R_High_Density_Non_Contextual',
           
           'R10':'R_High_Density_Non_Contextual',
           'R10A':'R_High_Density_Non_Contextual',
           'R10X':'R_High_Density_Non_Contextual',
         

          }

df2["Zoning District2"] = df2["Zoning District2"].map(zonedic)
df2["Zoning District2"] =  df2["Zoning District2"].astype(str)

zoning = sorted(list(set(df2["Zoning District2"].tolist() )))


for i in range(0, len(df2)):
    if "/" in df2["Zoning District"][i]:
        df2["Zoning District2"][i] = "M_Light_R_Low_Medium_Density_Non_Contextual"


#zoning

In [ ]:
df3 = df2.copy()

subset = ['Job Number','BIN Number', "Borough Name", "Job Location House Number", "Job Location Street Name",   
          'Boiler', 'Mechanical', 'Plumbing', 'Standpipe', 'Construction Equipment', 'General Construction', 'Curb Cut',
         'Proposed Occupancy Classification Description',
         'Primary Structural System Description', 'Community Board','Zoning District2',
         'Proposed Lot Detail : Lot Type Description',
          'Enclosed Parking (Y/N)', 'Yard Flag (Y/N)', 
          'Stories Category', 'Applicant Name', 'Plan Exam Count', "EstFeeLogNorm", 
          "ZoningFloorLogNorm", "DwellUnitsLogNorm", "FARLogNorm"]

df3 = df3[subset]



In [ ]:
for col in df3.columns:
    print (col)

In [ ]:
df3.to_csv(path + "Long Duration Category Analysis.csv", index=False)

In [ ]:
df3.dtypes

In [ ]:
#those = sorted(list(set(df3["Proposed Lot Detail : Lot Type Description"].tolist() )))
#those

In [ ]:
#df2.to_csv(path + "test.csv", index=False)

### CREATE OUTCOME VARIABLE

In [ ]:
pd.set_option('chained_assignment', None)

len(df)

#df2["LongDur"] = 0

#for i in range(0, len(df)):
#    if df["bussDays"][i] > 30:
#        df["LongDur"][i] = 1

def LongDur(x):
    if x <= 5:
        return 0
    if x > 5:
        return 1

df3["Outcome"] = df3["Plan Exam Count"].apply(LongDur)
       
        
print ("len of df", len(df3))
print ("number of long dur", df3["Outcome"].sum())
print ("percent long dur", float(df3["Outcome"].sum())/len(df) )

In [ ]:
df3["Lot Area (sqft)"].describe()

In [ ]:
#df3.to_csv(path + "Plan Exam Count.csv", index=False)
df3.to_csv(path + "Long Duration Category Analysis.csv", index=False)

In [ ]:
df3.dtypes

### APPLICANT ANALYSIS

### RANDOM SORT AND UPSAMPLE

In [ ]:
dfinal=df3.copy()
#dfinal = dfinal.rename(columns={'Appointment Count': 'Plan Exam Count'})


"""
from sklearn.utils import resample
# Separate majority and minority classes
df_majority = df2[df2.Outcome==0]
df_minority = df2[df2.Outcome==1]

print ("Size of majority class", len(df_majority) )
print ("Size of minority class", len(df_minority) )

lenMajority = len(df_majority)


# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=lenMajority, # to match majority class
                                 random_state=123) # reproducible results

print ( "Size of minority class upsampled", len(df_minority_upsampled) )

# Combine majority class with upsampled minority class
df2 = pd.concat([df_majority, df_minority_upsampled])
df2 = df2.reset_index(drop=True)
 
print ( "Display new class counts" )
df2.Outcome.value_counts()

df2 = df2.fillna(0)

print ("Display new class counts" )
df2.Outcome.value_counts()

"""
print ("Randomly sort dfinal" )
dfinal["Index"] = dfinal.index
dfinal = dfinal.sample(frac=1)
dfinal = dfinal.reset_index(drop=True)

print ("Size of final dataset ", len(dfinal) )




### MAKE DUMMIES AND BUILD MODEL

In [ ]:
#df3.to_csv(path + "Long Job Duration BIS 3.csv")

In [ ]:
#df["EstFeeCat"] = df["J_TOTAL_ESTIMATED_FEE"].apply(EstimatedFee)
categories = set(df["FARCat"].tolist() )
print (categories)


In [ ]:
#dfinal = df3.copy()


#dfinal = dfinal.drop('Enclosed Parking (Y/N)', 1)
#dfinal = dfinal.drop('Yard Flag (Y/N)', 1)
#dfinal = dfinal.drop('Plan Examiner BIS Name', 1)

def MakeDummies(df):
    print ("Parking and yard flags")
    df["Enclosed Parking (Y/N)"] = df["Enclosed Parking (Y/N)"].str.replace('Y', '1')
    df["Enclosed Parking (Y/N)"] = df["Enclosed Parking (Y/N)"].str.replace('N', '0')
    df["Enclosed Parking (Y/N)"] = df["Enclosed Parking (Y/N)"].astype(int)

    df["Yard Flag (Y/N)"] = df["Yard Flag (Y/N)"].str.replace('Y', '1')
    df["Yard Flag (Y/N)"] = df["Yard Flag (Y/N)"].str.replace('N', '0')
    df["Yard Flag (Y/N)"] = df["Yard Flag (Y/N)"].astype(int)
    
    """
    print ("Estimated fee")
    dummies = pd.get_dummies(df["EstFeeCat"],prefix='F' )
    print (len(dummies), len(df))
    df = pd.concat([df, dummies], axis=1)
    df = df.drop('EstFeeCat', 1)
    df = df.drop('J_TOTAL_ESTIMATED_FEE', 1)
    
    print ("Proposed zoning floor area")
    dummies = pd.get_dummies(df["FloorAreaCat"],prefix='FA' )
    print (len(dummies), len(df))
    df = pd.concat([df, dummies], axis=1)
    df = df.drop('FloorAreaCat', 1)
    df = df.drop('Proposed Zoning Floor area', 1)
    
    print ("Proposed dwelling units")
    dummies = pd.get_dummies(df["DwellingUnitCat"],prefix='DU' )
    print (len(dummies), len(df))
    df = pd.concat([df, dummies], axis=1)
    df = df.drop('DU_0', 1)
    df = df.drop('DwellingUnitCat', 1)
    df = df.drop('Proposed Dwelling Units', 1)
    
    print ("FAR")
    dummies = pd.get_dummies(df["FARCat"],prefix='FAR' )
    print (len(dummies), len(df))
    df = pd.concat([df, dummies], axis=1)
    df = df.drop('FARCat', 1)
    df = df.drop('Floor Area Ratio (FAR)  Floor Area', 1)
    
    #df["FARCat"] = df["Floor Area Ratio (FAR)  Floor Area"].apply(FAR)
    """
    

    print ("Occupancy classification")
    dummies = pd.get_dummies(df["Proposed Occupancy Classification Description"],prefix='O' )
    print (len(dummies), len(df))
    df = pd.concat([df, dummies], axis=1)
    df = df.drop('Proposed Occupancy Classification Description', 1)

    print ("Building materials")
    dummies = pd.get_dummies(df["Primary Structural System Description"],prefix='S' )
    print (len(dummies), len(df))
    df = pd.concat([df, dummies], axis=1)
    df = df.drop('Primary Structural System Description', 1)

    print ("Zoning")
    dummies = pd.get_dummies(df["Zoning District2"],prefix='Z' )
    dummies = dummies.drop('Z_nan', 1)
    print (len(dummies), len(df))
    df = pd.concat([df, dummies], axis=1)
    df = df.drop('Zoning District2', 1)

    print ("Lot detail")
    dummies = pd.get_dummies(df["Proposed Lot Detail : Lot Type Description"],prefix='L' )
    print (len(dummies), len(df))
    df = pd.concat([df, dummies], axis=1)
    df = df.drop('Proposed Lot Detail : Lot Type Description', 1)


    print ("Stories category")
    dummies = pd.get_dummies(df["Stories Category"] )
    print (len(dummies), len(df))
    df = pd.concat([df, dummies], axis=1)
    df = df.drop('Stories Category', 1)

    """
    print ("Applicants")
    dfapp = df[[ "Applicant Name", "Appointment Count"] ]
    dfapp["Applicant Name"] = dfapp["Applicant Name"].map(str.strip)
    dfapp["Job Count"] = 1
    dfapp = dfapp.groupby(['Applicant Name']).sum()
    dfapp = dfapp.add_suffix('').reset_index()

    #print (dfapp["Job Count"].describe())

    dfapp["Average Appointment Count"] = dfapp["Appointment Count"]/dfapp["Job Count"]

    applicantKeepList = []

    for i in range(0, len(dfapp)):
        if dfapp["Job Count"][i] > 10:
            applicantKeepList.append(dfapp["Applicant Name"][i])

    dummies = pd.get_dummies(df["Applicant Name"])
    print ("dummies shape:", dummies.shape)
    dummies = dummies[applicantKeepList]
    print ("dummies shape:", dummies.shape)
    df = pd.concat([df, dummies], axis=1)
    """



    df = df.drop('Applicant Name', 1)

    #print ("df shape:",df.shape )



    print ("Community board")
    df["Community Board"] = df["Community Board"].astype(str)
    df["Community Board"] = df["Community Board"].str.split(".").str[0]
    dfcb = df[[ "Community Board"] ]
    dfcb["Community Board"] = dfcb["Community Board"].astype(str).map(str.strip)
    dfcb["Job Count"] = 1
    dfcb = dfcb.groupby(['Community Board']).sum()
    dfcb = dfcb.add_suffix('').reset_index()

    #dfcb["Average Appointment Count"] = dfcb["Plan Exam Count"]/dfcb["Job Count"]
    cbKeepList = []
    for i in range(0, len(dfcb)):
        if dfcb["Job Count"][i] > 10:
            cbKeepList.append(dfcb["Community Board"][i])

    dummies = pd.get_dummies(df["Community Board"])
    print ("dummies shape:", dummies.shape)
    dummies = dummies[cbKeepList]
    print ("dummies shape:", dummies.shape)        
    df = pd.concat([df, dummies], axis=1)
    df = df.drop('Community Board', 1)
    print ("df shape:",df.shape )

    df = df.drop('Plan Exam Count', 1)
    
    return df




dfinal = MakeDummies(dfinal)
#print ()
#print ("X_test shape:", dfinal.shape)


# MODEL SELECTION

In [ ]:
def DropVars(df):
    df = df.drop('Index', 1)
    df = df.drop('Job Number', 1)
    df = df.drop('BIN Number', 1)
    df = df.drop('Borough Name', 1)
    df = df.drop('Job Location House Number', 1)
    df = df.drop('Job Location Street Name', 1)
    
    #df = df.drop('Proposed Zoning Floor area', 1)
    #df = df.drop('Lot Area (sqft)', 1)

    
    return df

In [ ]:

import pandas as pd
import numpy as np
import math

    
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


from sklearn.model_selection import train_test_split
#from adspy_shared_utilities import plot_class_regions_for_classifier_subplot
#create data
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_blobs
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

    
model_selection = []
    
def AccuracyAndAUC(model, clf, X_test, y_test, a, b):
    a = "(" + a + ","
    b = b + ")"
    accuracy_param = a + b
    clf_predicted = clf.predict(X_test)
        
    pred_prob = clf.predict_proba(X_test)[:,1]
    roc=roc_auc_score(y_test, pred_prob)
    #print ('AUC:', round(roc, 2))

        
    #clf_predicted = clf.predict(X_test)
    #confusion = confusion_matrix(y_test, clf_predicted)

    #print('Classifier Confusion Matrix\n', confusion)
    #print()
        
    model_selection.append([model,accuracy_param, clf.score(X_train, y_train), accuracy_score(y_test, clf_predicted),
                               precision_score(y_test, clf_predicted),recall_score(y_test, clf_predicted), 
                                f1_score(y_test, clf_predicted), roc])
        
        
    

y = dfinal["Outcome"]
X = dfinal.drop('Outcome', 1)
#X = X.drop('Bin', 1)
#X = X.drop('CB Number', 1)

#Create training, validation and test datasets
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
lengthdf2 = int(len(df2))
lengthdf2_20perc = int(lengthdf2*0.20)
lengthdf2_60perc = int(lengthdf2*0.60)

print (lengthdf2_60perc)
print (lengthdf2_20perc)

y_train = y[0:lengthdf2_60perc]
X_train = X[0:lengthdf2_60perc]
X_train = DropVars(X_train)

y_test = y[lengthdf2_60perc:lengthdf2_60perc + lengthdf2_20perc]
X_test = X[lengthdf2_60perc:lengthdf2_60perc + lengthdf2_20perc]
X_test = DropVars(X_test)

X_hold = X[lengthdf2_60perc + lengthdf2_20perc:]
y_hold = y[lengthdf2_60perc + lengthdf2_20perc:]

print ("Make original data for holdout to merge later with holdout results")
holdOrigData = df3[lengthdf2_60perc + lengthdf2_20perc:]


#test baseline using dummy classifier
dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)
    
print ("Baseline Accuracy: Dummy majority score:", round(dummy_majority.score(X_test, y_test), 2) )
print ()

print ("IMPLEMENT GBT MODELS")


#GRADIENT BOOSTED DECISION TREES
print ("GRADIENT BOOSTED DECISION TREES")
print ("Gradient boosting default parameters")
clf = GradientBoostingClassifier(random_state = 0)
clf.fit(X_train, y_train)
model = "Gradient Boosted Dec Trees"
a = "default"
b = "default"
AccuracyAndAUC(model, clf, X_test, y_test, a, b)
#PlotROC(clf, X_test, y_test)




#print ()
print ("GBT")
print ("Adjusting gradient boosted learning rate and depth")
learn_rate = [0.01, 0.04, 0.06, 0.08, 0.1, 0.2,0.4, 0.6]
max_depth = [2,4, 6]
    
for learn in learn_rate:
    for depth in max_depth:
        clf = GradientBoostingClassifier(learning_rate = learn, max_depth = depth, random_state = 0)
        clf.fit(X_train, y_train)
        a = "learning rate:" + str(learn)
        b = "maximum depth:" + str(depth)
        AccuracyAndAUC(model, clf, X_test, y_test, a, b)
"""

print ("Neural networks")
#hidden_layers = [(10), (20), (30), (40), (50), (60), (70), (80), (90), (100) ]

#hidden_layers = [(100,100,100) ]

hidden_layers = [(10), (20), (30), (40), (50), (60), (70), (80), (90), (100), (150), (200),
                 (10,10), (20,20), (30,30), (40,40), 
                 (50,50), (60,60), (70,70),
                 (10,10,10), (20,20,20), (30,30,30), 
                 (40,40,40), (50,50,50), (60,60,60), (70,70,70)]

#hidden_layers = [(20),  (40),(60), (80), (100), (120), (140), (160), (180), (200),
#                 (20,20), (40,40), (60,60), (80,80), (100,100), (120,120), (140,140), (160,160), (180, 180), (200,200),
#                (20,20,20), (40,40,40), (60,60,60), (80,80,80), (100,100,100), (120,120,120), (140,140,140), (160,160,160),
#                (180,180, 180), (200, 200, 200), (250, 250, 250),(300, 300, 300)  ]

iterations = 1000

#NEURAL NETWORKS
print ("Neural Networks")

for layer in hidden_layers:
    clf = MLPClassifier(hidden_layer_sizes = layer, max_iter=iterations) 
    clf.fit(X_train, y_train)
    modelstr = "Neural Networks"
    a = str(layer)
    b = str(iterations)
    print (modelstr, a)
    AccuracyAndAUC(modelstr, clf, X_test, y_test, a, b)
       
"""     


df_model = pd.DataFrame(model_selection, columns=('Model','Parameters', 'Training Score', 'Test Score', 'Precision', 'Recall', 'F1', 'AUC'))



In [ ]:
df_model = df_model.sort_values(by = 'Recall', ascending=False).reset_index(drop=True)

df_model["Diff"] = df_model["Training Score"] - df_model["Test Score"]
#df_model = df_model.sort_values(by = 'Diff', ascending=True).reset_index(drop=True)


df_model

### FEATURE IMPORTANCE

In [ ]:
learnRate = 0.04
mDepth = 4

clf = GradientBoostingClassifier(learning_rate = learnRate, max_depth = mDepth, random_state = 0)
clf.fit(X_train, y_train)

feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(X_train.columns, clf.feature_importances_):
    feats[feature] = importance #add the name/value pair 
    #print (feature, importance)
    
x = pd.DataFrame(list(feats.items()),columns = ['Feature','Importance']) 

x = x.sort_values(by = 'Importance', ascending=False).reset_index(drop=True)

#for i in range(0, len(x)):
#    print (x["Feature"][i], x["Importance"][i].round(3))

x2 = x[x["Importance"] > 0.01]

importantfeatures = x2["Feature"].tolist()


x[:30]

In [ ]:
df3.to_csv(path + "Long Duration Job FINAL.csv", index=False)

### ANALYSIS ON HOLDOUT

In [ ]:
X_train2 = X_train.copy()
y_train2 = y_train.copy()

y_hold2 = y_hold.copy()
X_hold2 = X_hold.copy()
X_hold2 = DropVars(X_hold2)
#X_hold2 = X_hold2.drop('Index', 1)
#X_hold2 = X_hold2.drop('Job Number', 1)
#X_hold2 = X_hold2.drop('BIN Number', 1)
#X_hold2 = X_hold2.drop('Borough Name', 1)
#X_hold2 = X_hold2.drop('Job Location House Number', 1)
#X_hold2 = X_hold2.drop('Job Location Street Name', 1)

#X_hold2 = DropFields(X_hold2)
print ("Training shape",X_train2.shape )
#X_hold_raw = X_hold.copy()
#X_hold2 = X_hold.copy()
#X_hold = df_hold.copy()
#y_hold2 = X_hold2["Outcome"]
#X_hold2 = X_hold2.drop('Outcome', 1)
#print ("Reducing number of features")
#X_hold2 = X_hold2[importantfeatures]
print ("Holdout shape",X_hold2.shape )


#Gradient Boosted Trees
clf = GradientBoostingClassifier(learning_rate = learnRate, max_depth = mDepth, random_state = 0)
#clf = MLPClassifier(hidden_layer_sizes = (70), max_iter=1000) 


clf.fit(X_train2, y_train2)

from sklearn.metrics import confusion_matrix
clf_predicted = clf.predict(X_hold2)
confusion = confusion_matrix(y_hold2, clf_predicted)
print ("Confusion Matrix")
print (confusion)


print ("Calculate predictions using best model")
result  =   clf.predict_proba(X_hold2)
#assign positive class probability predictions
    
X_hold2["PredProb"] = result[:,1]

X_hold2 = pd.concat([X_hold2, y_hold2], axis=1)

print ("create predicted outcome")
X_hold2["PredOutcome"] = 0

X_hold2["Index"] = X_hold2.index
X_hold2 = X_hold2.reset_index(drop=True)

for i in range(0, len(X_hold2)):
    if X_hold2["PredProb"][i] >= 0.5:
        X_hold2["PredOutcome"][i] = 1
        
print ("DONE")

#print (clf)
print ("Size of final test data (holdout data):", len(X_hold2))
print ("True positive:", sum(confusion[1]) )
print ("True negative:", sum(confusion[0]) )
print ("Correct Predictions:",confusion[1][1] )
print ("Percent correctly predicted:",round(float(confusion[1][1])/sum(confusion[1]), 2) )
print ("False negatives:", confusion[1][0])
print ("Percent false negatives:",  round(confusion[1][0]/sum(confusion[1]),2) )
print ("False positives:", confusion[0][1] )
print ("Percent false positives:", round(float(confusion[0][1])/len(X_hold2), 2) )



In [ ]:
print ("Merge holdout datasets with predicted outcome with one with job number")
print ("X Holdout")
print (holdOrigData.shape, X_hold2.shape)
#print ("Y Holdout")
#print (y_hold.shape, y_hold2.shape)
mergeX_hold2 = X_hold2[["PredProb", "Outcome", "PredOutcome","Index"]]
print (mergeX_hold2.shape)
print (holdOrigData.shape)

#output_holdout = X_hold.copy()
holdOrigData = holdOrigData.reset_index(drop=True)
output_holdout = pd.concat([holdOrigData, mergeX_hold2], axis=1)

print (output_holdout.shape)

output_holdout = output_holdout.drop_duplicates(['Job Number']).reset_index(drop=True)

print (output_holdout.shape)


#output_holdout.to_csv(path + "Holdout Dataset Model Predictions.csv", index=False)

In [ ]:
#output_holdout.to_csv(path + "Long Dur Pred Holdout.csv", index=False)
X_hold2.to_csv(path + "Long Dur Pred Holdout.csv", index=False)

In [ ]:
output_holdout

In [ ]:
#dfinal.dtypes[:50], dfinal.dtypes[50:100], dfinal.dtypes[100:150], dfinal.dtypes[150:]